### **Итоговое задание Альмира и Соколова Александра**  
### по Проекту 6. Car Price prediction 
####  Юнит 6. Основные алгоритмы машинного обучения. Часть II (отредактирован 09.09.2020)
---

### 1. Импорт библиотек

In [31]:
import pandas as pd
import requests, json
import time
import os
print(os.listdir('./data'))
PATH_to_file = './data/'

['all_car_brands_and_their_models_from_auto_ru.csv', 'test.csv', 'test_auto_ru.csv']


### 2. Импорт данных о марках и моделях

In [32]:
df_marks_and_models = pd.read_csv(PATH_to_file+'all_car_brands_and_their_models_from_auto_ru.csv', index_col=0)
print('Кол-во марок: ', df_marks_and_models.shape[0])
display(df_marks_and_models.head(2))

Кол-во марок:  262


,Models
Mark,
AC,"['378_GT', 'ACE', 'ACECA', 'COBRA']"
ACURA,"['CL', 'CSX', 'EL', 'ILX', 'INTEGRA', 'LEGEND'..."


### 2. Отбор марок для парсинга

In [33]:
# кол-во объявлений на сайте на 9.9.2020 
dict_of_num_cars_of_models ={'AC':2, 'Acura':94, 'Alfa Romeo':65, 'Alpina':10, 
                             'AMC':2, 'Ariel':1, 'Asia':1, 'Aston Martin':25, 
                             'Audi':4093, 'Austin Healey':1, 'Bajaj':3, 
                             'Bentley':104, 'BMW':7423, 'Borgward':1, 
                             'Brilliance':10, 'Buick':17, 'BYD':22, 
                             'Cadillac':321, 'Changan':100, 'Chery':593, 
                             'Chevrolet':3670, 'Chrysler':316, 'Citroen':1142, 
                             'Dacia':9, 'Daewoo':1024, 'Daihatsu':47, 
                             'Daimler':5, 'Datsun':212, 'Delage':1, 'Derways':2, 
                             'DKW':1, 'Dodge':413, 'DongFeng':25, 'Doninvest':2, 
                             'DS':3, 'DW Hower':8, 'Eagle':1, 'Excalibur':1, 
                             'FAW':66, 'Ferrari':29, 'Fiat':206, 'Ford':4812, 
                             'Foton':2, 'GAC':14, 'Geely':302, 'Genesis':54, 
                             'GMC':26, 'Great Wall':420, 'Hafei':6, 'Haima':12, 
                             'Haval':110, 'Hawtai':4, 'Heinkel':1, 'Honda':1179, 
                             'HuangHai':1, 'Hummer':70, 'Hyundai':5712, 
                             'Infiniti':1041, 'Innocenti':1, 'Iran Khodro':21, 
                             'Isuzu':43, 'JAC':8, 'Jaguar':308, 'Jeep':466, 'JMC':1, 
                             'Kia':5595, 'LADA (ВАЗ)':8935, 'Lamborghini':14, 
                             'Lancia':8, 'Land Rover':1616, 'Lexus':985, 
                             'Lifan':295, 'Lincoln':62, 'Lotus':1, 'Luxgen':9, 
                             'Mahindra':1, 'Maserati':54, 'Maybach':7, 
                             'Mazda':2311, 'McLaren':5, 'Mercedes-Benz':8005, 
                             'Mercury':16, 'Metrocab':2, 'MG':10, 'MINI':313, 
                             'Mitsubishi':3935, 'Mitsuoka':3, 'Nissan':5776, 
                             'Oldsmobile':10, 'Opel':3131, 'Packard':2, 
                             'Peugeot':1636, 'Plymouth':13, 'Pontiac':38, 
                             'Porsche':714, 'Proton':3, 'PUCH':1, 'Ravon':112, 
                             'Renault':2771, 'Rolls-Royce':55, 'Rover':69, 
                             'Saab':147, 'Saturn':9, 'Scion':9, 'SEAT':77, 
                             'Shanghai Maple':1, 'Skoda':2810, 'Smart':89, 
                             'SsangYong':718, 'Subaru':934, 'Suzuki':878, 
                             'Tatra':5, 'Tesla':43, 'Tianma':2, 'Toyota':4648, 
                             'Triumph':1, 'Vauxhall':1, 'Volkswagen':7109, 
                             'Volvo':2069, 'Vortex':91, 'Wartburg':1, 'Willys':2, 
                             'Zotye':15, 'ZX':4, 'ГАЗ':935, 'Гоночный болид':2, 
                             'ЗАЗ':231, 'ЗИЛ':2, 'ИЖ':129, 'ЛуАЗ':45, 
                             'Москвич':198, 'СМЗ':4, 'ТагАЗ':68, 'УАЗ':1107}

In [34]:
# уберем модели с кол-вом объявлений ниже 300 и повысим регистр индексов 
list_keys_to_delete = []
for mark in dict_of_num_cars_of_models.keys():
    if dict_of_num_cars_of_models[mark] < 300:
        list_keys_to_delete.append(mark)
    
for key in list_keys_to_delete: del dict_of_num_cars_of_models[key]

for mark in list(dict_of_num_cars_of_models.keys()):
    mark_up = mark.upper()
    dict_of_num_cars_of_models[mark_up]=dict_of_num_cars_of_models.pop(mark)

In [35]:
# посмотрим на список получившихся моделей
print(*dict_of_num_cars_of_models, sep=',')

AUDI,BMW,CADILLAC,CHERY,CHEVROLET,CHRYSLER,CITROEN,DAEWOO,DODGE,FORD,GEELY,GREAT WALL,HONDA,HYUNDAI,INFINITI,JAGUAR,JEEP,KIA,LADA (ВАЗ),LAND ROVER,LEXUS,MAZDA,MERCEDES-BENZ,MINI,MITSUBISHI,NISSAN,OPEL,PEUGEOT,PORSCHE,RENAULT,SKODA,SSANGYONG,SUBARU,SUZUKI,TOYOTA,VOLKSWAGEN,VOLVO,ГАЗ,УАЗ


In [36]:
# Удалим отечественные модели
for key in ['LADA (ВАЗ)','ГАЗ','УАЗ']: del dict_of_num_cars_of_models[key]

In [37]:
# проверим что названия моделей совпадаю с названиями каталога сайта auto.Ru
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in dict_of_num_cars_of_models.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего марок с кол-вом объявлений более 300:= {len(dict_of_num_cars_of_models)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего марок с кол-вом объявлений более 300:= 36
список марок, которые нужно проверить := GREAT WALL LAND ROVER MERCEDES-BENZ SSANGYONG


In [38]:
dict_of_num_cars_of_models['GREAT WALL']

420

In [39]:
# исправим несоответствия
dict_of_num_cars_of_models['GREAT_WALL']=dict_of_num_cars_of_models.pop('GREAT WALL')
dict_of_num_cars_of_models['LAND_ROVER']=dict_of_num_cars_of_models.pop('LAND ROVER')
dict_of_num_cars_of_models['MERCEDES']=dict_of_num_cars_of_models.pop('MERCEDES-BENZ')
dict_of_num_cars_of_models['SSANG_YONG']=dict_of_num_cars_of_models.pop('SSANGYONG')

In [40]:
# еще раз проверим что названия моделей совпадаю с названиями каталога сайта auto.Ru
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in dict_of_num_cars_of_models.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего марок с кол-вом объявлений более 300 := {len(dict_of_num_cars_of_models)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего марок с кол-вом объявлений более 300 := 36
список марок, которые нужно проверить :=


In [41]:
# запишем список релевантных марок для парсинга
list_of_relevant_marks = list(dict_of_num_cars_of_models.keys())

In [42]:
sum_of_ads = sum(dict_of_num_cars_of_models.values())
print(f"примерное кол-во строк после парсинга должно быть := {sum_of_ads}")

примерное кол-во строк после парсинга должно быть := 89189


### 3. Подготовка к парсингу

In [43]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Content-Length': '137',
    'content-type': 'application/json',
    'Cookie': '_csrf_token=1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24; autoru_sid=a%3Ag5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270%7C1580931467355.604800.8HnYnADZ6dSuzP1gctE0Fw.cd59AHgDSjoJxSYHCHfDUoj-f2orbR5pKj6U0ddu1G4; autoruuid=g5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270; suid=48a075680eac323f3f9ad5304157467a.bc50c5bde34519f174ccdba0bd791787; from_lifetime=1580933172327; from=yandex; X-Vertis-DC=myt; crookie=bp+bI7U7P7sm6q0mpUwAgWZrbzx3jePMKp8OPHqMwu9FdPseXCTs3bUqyAjp1fRRTDJ9Z5RZEdQLKToDLIpc7dWxb90=; cmtchd=MTU4MDkzMTQ3MjU0NQ==; yandexuid=1758388111580931457; bltsr=1; ',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202002.03.092255',
    'x-client-date': '1580933207763',
    'x-csrf-token': '1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24',
    'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
    'x-requested-with': 'fetch'
}

In [44]:
url = "https://auto.ru/-/ajax/desktop/listing/"

In [45]:
# посмотрим какие столбцы нам нужны 
test = pd.read_csv(PATH_to_file+'test.csv')
print('Кол-во столбцов теста:', test.shape[1])
print('Названия столбцов теста:', list(test.columns))

# вывод всех столбцов датафрейма
pd.set_option('display.max_columns', None)
test.head(2)

Кол-во столбцов теста: 23
Названия столбцов теста: ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'engineDisplacement', 'enginePower', 'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние', 'Владельцы', 'ПТС', 'Таможня', 'Владение', 'id']


,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id
0,седан,BMW,чёрный,дизель,2016.0,520d 2.0d AT (190 л.с.),4.0,2017.0,SEDAN AUTOMATIC 2.0,автоматическая,2.0 LTR,190 N12,В РОЛЬФ Ясенево представлено более 500 автомоб...,158836.0,"['[{""name"":""Безопасность"",""values"":[""Антипробу...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,0
1,седан,BMW,белый,дизель,2018.0,318d 2.0d AT (150 л.с.),4.0,2019.0,SEDAN AUTOMATIC 2.0,автоматическая,2.0 LTR,150 N12,Автомобиль из демонстрационного парка по спец ...,10.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,1


In [46]:
# сделаем тестовый вызов запроса для BMW модели M6 и выведем его на экран, чтобы по нему можно было сделать функцию для создания 23 необходимых столбцов
model = 'M6'
param = {
                'catalog_filter' : [{"mark": 'BMW', "model": model}],
                'section': "used",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"600",
                "geo_radius":200,
                "geo_id":[213],
                'page': 1
            }

response = requests.post(url, json=param, headers = HEADERS)
data = response.json()
for elem in data['offers']:
    print(elem)
    break

{'availability': 'IN_STOCK', 'category': 'cars', 'color_hex': '97948F', 'delivery_info': {}, 'description': 'Состояние нового авто. Использовалась летом в качестве машины выходного дня. Оригинальный пробег. Обслуживание у официального дилера. Новая резина. Состояние автомобиля гарантирую. ', 'documents': {'owners_number': 2, 'pts_original': True, 'custom_cleared': True, 'year': 2015, 'vin': 'WBS**************', 'pts': 'ORIGINAL', 'license_plate': '******|777', 'vin_resolution': 'OK'}, 'is_favorite': False, 'old_category_id': 15, 'owner_expenses': {'transport_tax': {'tax_by_year': 84000, 'year': 2020, 'rid': 213, 'rate': 150, 'horse_power': 560, 'holding_period_month': 12, 'boost': 1}, 'osago_insurance': [{'insurance_company': 'TINKOFF', 'price': 6309}]}, 'section': 'used', 'seller_type': 'PRIVATE', 'services': [{'service': 'all_sale_special', 'is_active': True, 'prolongable': False}, {'service': 'all_sale_toplist', 'is_active': True, 'prolongable': False}, {'service': 'all_sale_color',

In [47]:
# строка для проверки содержания элемента запроса
elem['additional_info']['hidden']

False

In [48]:
def utils_clean_values_dict(d_dict):
    for elem in d_dict:
        d_dict[elem] = 0
    return

In [49]:
dict_columns = {'bodyType':0, 'brand':0, 'color':0, 'fuelType':0, 'modelDate':0, 
                'name':0, 'numberOfDoors':0, 'productionDate':0, 
                'vehicleConfiguration':0, 'vehicleTransmission':0, 
                'engineDisplacement':0, 'enginePower':0, 'description':0, 
                'mileage':0, 'Комплектация':0, 'Привод':0, 'Руль':0, 
                'Состояние':0, 'Владельцы':0, 'ПТС':0, 'Таможня':0, 
                'Владение':0, 'price':0}

model = 'M6'
# создаем функцию для обработки 23 необходимых столбцов
def from_elem_to_dict(d_elem, d_dict):
    utils_clean_values_dict(d_dict)
    
    try: d_dict['bodyType'] = d_elem['vehicle_info']['configuration']['human_name']
    except: d_dict['bodyType'] = None

    try: d_dict['brand'] = d_elem['vehicle_info']['mark_info']['code']
    except: d_dict['brand'] = None

    try: d_dict['color'] = d_elem['color_hex']
    except: d_dict['color'] = None

    try: d_dict['fuelType'] = d_elem['lk_summary'].split()[-1]
    except: d_dict['fuelType'] = None

    try: d_dict['modelDate'] = d_elem['vehicle_info']['super_gen']['year_from']
    except: d_dict['modelDate'] = None

    try: d_dict['name'] =  d_elem['vehicle_info']['tech_param']['human_name']
    except: d_dict['name'] = None

    try: d_dict['numberOfDoors'] = d_elem['vehicle_info']['configuration']['doors_count']
    except: d_dict['numberOfDoors'] = None

    try: d_dict['productionDate'] = d_elem['documents']['year']
    except: d_dict['productionDate'] = None

    try: d_dict['vehicleTransmission'] = d_elem['vehicle_info']['tech_param']['transmission']
    except: d_dict['vehicleTransmission'] = None

    try: d_dict['engineDisplacement'] = d_elem['vehicle_info']['tech_param']['human_name'].split()[0]
    except: d_dict['engineDisplacement'] = None

    try: d_dict['vehicleConfiguration'] = elem['vehicle_info']['configuration']['body_type']+' '+d_dict['vehicleTransmission']+' '+d_dict['engineDisplacement']
    except: d_dict['vehicleConfiguration'] = None

    try: d_dict['enginePower'] = d_elem['vehicle_info']['tech_param']['power']
    except: d_dict['enginePower'] = None

    try: d_dict['description'] = d_elem['description']
    except: d_dict['description'] = None

    try: d_dict['mileage'] = d_elem['state']['mileage']
    except: d_dict['mileage'] = None

    try: d_dict['Комплектация'] = d_elem['vehicle_info']['complectation']
    except: d_dict['Комплектация'] = None

    try: d_dict['Привод'] = d_elem['lk_summary'].split(', ')[-2]
    except: d_dict['Привод'] = None

    try: d_dict['Руль'] = d_elem['vehicle_info']['steering_wheel']
    except: d_dict['Руль'] = None

    # 'Состояние' не найдено
    d_dict['Состояние'] = None

    try: d_dict['Владельцы'] = d_elem['documents']['owners_number']
    except: d_dict['Владельцы'] = None

    try: d_dict['ПТС'] = d_elem['documents']['pts']
    except: d_dict['ПТС'] = None

    try: d_dict['Таможня'] = d_elem['documents']['custom_cleared']
    except: d_dict['Таможня'] = None

    try: d_dict['Владение'] =  d_elem['documents']['purchase_date']
    except: d_dict['Владение'] = None


    try: d_dict['price'] = d_elem['price_info']['RUR']
    except: d_dict['price'] = None

    # дата размещения объявления решил добавить 
    try: d_dict['start_date'] =  d_elem['additional_info']['hot_info']['start_time']
    except: d_dict['start_date'] = None

    # статус объявления 
    try: d_dict['hidden'] =  d_elemelem['additional_info']['hidden']
    except: d_dict['hidden'] = None

    # модель 
    d_dict['model'] =  model

    return

from_elem_to_dict(elem, dict_columns)

In [50]:
display(pd.DataFrame(dict_columns).head(2))
test[test['enginePower']=='560 N12'].head(2)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
id,Купе,BMW,97948F,бензин,2012,4.4 AMT (560 л.с.),2,2015,COUPE ROBOT 4.4,ROBOT,4.4,560,Состояние нового авто. Использовалась летом в ...,29000,0,задний,LEFT,None,2,ORIGINAL,True,None,3600000,2020-07-27T12:12:35Z,None,M6


,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id
168,купе,BMW,синий,бензин,2012.0,4.4 AMT (560 л.с.),2.0,2012.0,COUPE ROBOT 4.4,роботизированная,4.4 LTR,560 N12,РОЛЬФ Премиум – крупнейший официальный дилер М...,61375.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",задний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,168
276,седан,BMW,синий,бензин,2011.0,4.4 AMT (560 л.с.),4.0,2016.0,SEDAN ROBOT 4.4,роботизированная,4.4 LTR,560 N12,"По ПТС два владельца, по факту один",80250.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",задний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,2 года и 1 месяц,276


### 4. Парсинг

In [53]:
auto_ru = {}

iter_of_ads, iter_of_marks, old_iter_of_ads, sum_ads, sum_ads_from_dict = 0,0,0,0,0
space = ' '
start_time_global = time.clock()

for mark in list_of_relevant_marks:
    start_time = time.clock()
    list_of_models = json.loads(df_marks_and_models.loc[mark, 'Models'].replace("'", '"'))
    for model in list_of_models:
        for num_page in range(1, 200):
            param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"900",
                "geo_radius":200,
                "geo_id":[213],
                'page': num_page
            }
            response = requests.post(url, json=param, headers = HEADERS)
            data = response.json()

            if data['offers'] != []:
                for elem in data['offers']:
                    from_elem_to_dict(elem, dict_columns)
                    auto_ru[iter_of_ads] = dict_columns.copy()
                    iter_of_ads += 1
            else:
                break
    iter_of_marks +=1
    temp_1 = iter_of_ads-old_iter_of_ads
    temp_2 = dict_of_num_cars_of_models[mark]
    temp_3 = temp_1/temp_2
    temp_4 = space*abs(13-len(mark)-len(str(iter_of_marks)))
    temp_5 = int(time.clock() - start_time)
    temp_8 = round(temp_3*100,2)
    temp_7 = space*abs(18-len(str(temp_1))-len(str(temp_2))-len(str(temp_8))-len(str(temp_5)))
    temp_9 = int(time.clock() - start_time_global)
    

    sum_ads += temp_1
    sum_ads_from_dict += temp_2
    temp_def = sum_ads/sum_ads_from_dict
    temp_6 = (sum_of_ads*temp_def - iter_of_ads)*temp_9/sum_ads


    print (f'{iter_of_marks}.{mark} {temp_4} ушло:{temp_5} сек., стр. {temp_1} из {temp_2} {temp_8}%,{temp_7}всего стр. {iter_of_ads}, прогноз {int(temp_6/60)} мин.')

    old_iter_of_ads=iter_of_ads

print (f'Всего спарсили := {iter_of_ads} строк, на это ушло: {int((time.clock() - start_time_global)/60)} мин., ')

1.AUDI          ушло:205 сек., стр. 4084 из 4093 99.78%,  всего стр. 4084, прогноз 71 мин.
2.BMW           ушло:401 сек., стр. 7393 из 7423 99.6%,   всего стр. 11477, прогноз 68 мин.
3.CADILLAC      ушло:33 сек., стр. 322 из 321 100.31%,    всего стр. 11799, прогноз 69 мин.
4.CHERY         ушло:35 сек., стр. 566 из 593 95.45%,     всего стр. 12365, прогноз 69 мин.
5.CHEVROLET     ушло:149 сек., стр. 3633 из 3670 98.99%,  всего стр. 15998, прогноз 62 мин.
6.CHRYSLER      ушло:43 сек., стр. 323 из 316 102.22%,    всего стр. 16321, прогноз 64 мин.
7.CITROEN       ушло:112 сек., стр. 1146 из 1142 100.35%, всего стр. 17467, прогноз 66 мин.
8.DAEWOO        ушло:58 сек., стр. 1041 из 1024 101.66%,  всего стр. 18508, прогноз 65 мин.
9.DODGE         ушло:54 сек., стр. 418 из 413 101.21%,    всего стр. 18926, прогноз 67 мин.
10.FORD         ушло:242 сек., стр. 4797 из 4812 99.69%,  всего стр. 23723, прогноз 61 мин.
11.GEELY        ушло:19 сек., стр. 295 из 302 97.68%,     всего стр. 24018, прогн

In [54]:
# запись датасета
df_auto_ru = pd.DataFrame(auto_ru).T
df_auto_ru.to_csv(PATH_to_file+'all_auto_ru_09_09_2020.csv', index=False)
print(f'кол-во строк датасета:= {len(df_auto_ru)}')

кол-во строк датасета:= 89378


In [55]:
# визуальный контроль
df_auto_ru.head(3)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
0,Седан,AUDI,040001,бензин,1990,2.8 MT (174 л.с.) 4WD,4,1991,SEDAN MECHANICAL 2.8,MECHANICAL,2.8,174,"Машина в приличном состоянии ,не гнилая не р...",350000,{'id': '0'},полный,LEFT,None,3,ORIGINAL,True,None,200000,2019-10-03T08:09:11Z,None,100
1,Седан,AUDI,EE1D19,бензин,1982,1.8 MT (90 л.с.),4,1986,SEDAN MECHANICAL 1.8,MECHANICAL,1.8,90,Машина в оригинале не гнилая все вопросы по те...,173424,{'id': '0'},передний,LEFT,None,3,ORIGINAL,True,None,60000,2020-09-06T06:49:40Z,None,100
2,Универсал 5 дв.,AUDI,0000CC,бензин,1988,2.3 MT (136 л.с.) 4WD,5,1989,WAGON_5_DOORS MECHANICAL 2.3,MECHANICAL,2.3,136,ПТС Оригинал!\nПолный комплект ключей!\nПо рез...,230000,{'id': '0'},полный,LEFT,None,3,ORIGINAL,True,None,99000,2020-09-02T14:04:21Z,None,100


Первоначальный парсинг показал, что на сайте много старых объявлений . Так как изначально за последние 90 дней мы получили 43% от всех объявлений по BMW, поэтому пришлось скачать за последние 900 дней, чтобы получить полные цифры заявленные на сайте. 